In [1]:
from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import TensorBoard

In [2]:
# Define the log directory
log_dir = "logs/a1/"

# Create a TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [3]:
# Veri setinin bulunduğu klasörü belirtin
data_dir = 'lfw-deepfunneled/'

# Görüntüleri ve etiketleri depolamak için boş listeler oluşturun
images = []
labels = []

# Tüm klasörleri döngüye alarak görüntüleri yükleyin ve etiketleri oluşturun
for person_name in os.listdir(data_dir):
    person_dir = os.path.join(data_dir, person_name)
    if os.path.isdir(person_dir):
        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            images.append(image)
            labels.append(person_name)


In [4]:
class_names = np.unique(labels)
class_indices = {class_name: i for i, class_name in enumerate(class_names)}

# Etiketleri sayısal forma dönüştürme
labels = [class_indices[label] for label in labels]


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)


In [7]:
import numpy as np

X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)


In [6]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(250, 250, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5749, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 248, 248, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 124, 124, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 122, 122, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 61, 61, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 59, 59, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 29, 29, 128)      0

In [8]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[tensorboard_callback])


Epoch 1/10
331/331 [==============================] - 103s 310ms/step - loss: 8.7605 - accuracy: 0.0354 - val_loss: 8.0697 - val_accuracy: 0.0427
Epoch 2/10
331/331 [==============================] - 103s 312ms/step - loss: 7.2178 - accuracy: 0.0456 - val_loss: 8.4162 - val_accuracy: 0.0438
Epoch 3/10
331/331 [==============================] - 112s 340ms/step - loss: 5.2592 - accuracy: 0.0910 - val_loss: 10.6675 - val_accuracy: 0.0287
Epoch 4/10
331/331 [==============================] - 89s 268ms/step - loss: 2.0311 - accuracy: 0.5454 - val_loss: 16.6844 - val_accuracy: 0.0280
Epoch 5/10
331/331 [==============================] - 91s 276ms/step - loss: 0.4465 - accuracy: 0.8935 - val_loss: 24.0505 - val_accuracy: 0.0174
Epoch 6/10
331/331 [==============================] - 89s 268ms/step - loss: 0.1294 - accuracy: 0.9708 - val_loss: 26.4985 - val_accuracy: 0.0314
Epoch 7/10
331/331 [==============================] - 93s 280ms/step - loss: 0.0745 - accuracy: 0.9832 - val_loss: 30.5938 

In [9]:
from sklearn.metrics import accuracy_score

y_pred_train = model.predict(X_train)
y_pred_train = np.argmax(y_pred_train, axis=1)

train_accuracy = accuracy_score(y_train, y_pred_train)
print(f'Eğitim Doğruluk: {train_accuracy}')


331/331 [==============================] - 4s 10ms/step
Eğitim Doğruluk: 0.9833742679010014


In [10]:
y_pred = model.predict(np.array(X_val))
predicted_labels = np.argmax(y_pred, axis=1)


83/83 [==============================] - 1s 10ms/step


In [1]:
%load_ext tensorboard
%tensorboard --logdir=logs --port=6007



Reusing TensorBoard on port 6007 (pid 16976), started 1 day, 17:04:31 ago. (Use '!kill 16976' to kill it.)